# 1、检查GPU状态

In [10]:
import torch
import subprocess, os, platform, psutil

print("==== 系统信息 ====")
print(platform.platform())

print("\n==== GPU 信息（nvidia-smi）====")
try:
    subprocess.run(["nvidia-smi"])
except FileNotFoundError:
    print("⚠️ 系统未检测到 NVIDIA GPU 或未安装 nvidia-smi")

print("\n==== PyTorch CUDA 检查 ====")
print("PyTorch 版本:", torch.__version__)
print("CUDA 是否可用:", torch.cuda.is_available())
print("GPU 数量:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU 名称:", torch.cuda.get_device_name(0))
    print("显存信息:", torch.cuda.get_device_properties(0))

print("\n==== CPU / 内存 ====")
print("CPU 核心数:", psutil.cpu_count())
print("CPU 物理核心数:", psutil.cpu_count(logical=False))

mem = psutil.virtual_memory()
print(f"内存总量: {mem.total/1024/1024/1024:.2f} GB")
print(f"内存已使用: {mem.used/1024/1024/1024:.2f} GB")
print(f"内存可用: {mem.available/1024/1024/1024:.2f} GB")

print("\n==== 当前 Notebook 进程 ====")
print("PID:", os.getpid())


==== 系统信息 ====
Linux-5.10.134-18.0.4.lifsea8.x86_64-x86_64-with-glibc2.35

==== GPU 信息（nvidia-smi）====
Wed Nov 26 10:23:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10                     Off |   00000000:00:08.0 Off |                    0 |
|  0%   27C    P8             10W /  150W |       4MiB /  23028MiB |      0%      Default |
|                                    

# 2、导入数据

In [11]:
from sklearn.model_selection import KFold
from autogluon.tabular import TabularPredictor
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import pickle


import shutil
import os
os.chdir("/mnt/workspace")
print("当前工作目录:", os.getcwd())


warnings.filterwarnings('ignore')

当前工作目录: /mnt/workspace


In [12]:
class CFG:
    train_path = 'data/train.csv'
    test_path = 'data/test.csv'
    sample_sub_path = 'data/sample_submission.csv'

    target = 'Premium Amount'
    n_folds = 10
    seed = 42

    # 模型路径全部改为相对路径
    model_path = "model/autogluon_model"
    model_backup_path = "model/autogluon_model_backup"

    time_per_round = 3600
    n_rounds = 6

    presets = 'best_quality'
    excluded_model_types = ['KNN']
    ag_args_fit = {'num_gpus': 1, 'num_cpus': 8}


In [13]:
train = pd.read_csv(CFG.train_path, index_col='id')
test = pd.read_csv(CFG.test_path, index_col='id')

train[CFG.target] = np.log1p(train[CFG.target])

In [14]:
kf = KFold(n_splits=CFG.n_folds, random_state=CFG.seed, shuffle=True)
split = kf.split(train, train[CFG.target])
for i, (_, val_index) in enumerate(split):
    train.loc[val_index, 'fold'] = i

# 3、进行训练（断点续算）

In [15]:
# 备份的功能函数，用于进行断点续算
def backup_model_to_drive():
    src = CFG.model_path
    dst = CFG.model_backup_path
    if not os.path.exists(src):
        print("本地模型目录不存在，跳过备份。")
        return
    # 删掉旧备份，避免越堆越多
    if os.path.exists(dst):
        shutil.rmtree(dst)
    shutil.copytree(src, dst)
    print(f"[备份完成] 模型已保存到: {dst}")


In [16]:
# 如果 Drive 里已有备份，先恢复到本地 /content
if os.path.exists(CFG.model_backup_path):
    print("检测到 Drive 备份，恢复到本地继续训练...")
    if os.path.exists(CFG.model_path):
        shutil.rmtree(CFG.model_path)
    shutil.copytree(CFG.model_backup_path, CFG.model_path)
else:
    print("Drive 中暂无备份，本次视为全新训练。")

检测到 Drive 备份，恢复到本地继续训练...


In [17]:
from autogluon.tabular import TabularPredictor

predictor = None

for i in range(CFG.n_rounds):
    print(f"\n======== 第 {i+1}/{CFG.n_rounds} 段训练开始 ========")

    if os.path.exists(CFG.model_path):
        # 已有模型 → 断点续算
        if predictor is None:
            predictor = TabularPredictor.load(CFG.model_path)
        print("使用 fit_extra 继续训练...")
        predictor.fit_extra(
            hyperparameters='default',
            time_limit=CFG.time_per_round,
            excluded_model_types=CFG.excluded_model_types,
            ag_args_fit=CFG.ag_args_fit,
        )
    else:
        # 第一次训练
        print("首次训练，创建新模型...")
        predictor = TabularPredictor(
            path=CFG.model_path,   # 注意：这里现在是相对路径 "model/autogluon_model"
            problem_type='regression',
            eval_metric='rmse',
            label=CFG.target,
            groups='fold',
            verbosity=2
        )
        predictor.fit(
            train_data=train,
            time_limit=CFG.time_per_round,
            presets=CFG.presets,
            excluded_model_types=CFG.excluded_model_types,
            ag_args_fit=CFG.ag_args_fit,
        )

    backup_model_to_drive()
    print(f"======== 第 {i+1} 段训练结束，并已备份到 Drive ========")


Excluded models: ['KNN'] (Specified by `excluded_model_types`)
Fitting 9 L1 models ...
Fitting model: LightGBMXT_19_BAG_L1 ... Training model for up to 3600.0s of the 3599.99s of remaining time.



======== 第 1/6 段训练开始 ========
使用 fit_extra 继续训练...


	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0488	 = Validation score   (-root_mean_squared_error)
	954.62s	 = Training   runtime
	36.32s	 = Validation runtime
Fitting model: LightGBM_18_BAG_L1 ... Training model for up to 1793.11s of the 1793.1s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0456	 = Validation score   (-root_mean_squared_error)
	146.33s	 = Training   runtime
	2.41s	 = Validation runtime
Fitting model: RandomForestMSE_7_BAG_L1 ... Training model for up to 1644.71s of the 1644.71s of remaining time.
	-1.0472	 = Validation score   (-root_mean_squared_error)
	539.82s	 = Training   runtime
	32.16s	 = Validation runtime
Fitting model: CatBoost_7_BAG_L1 ... Training model for up to 1071.86s of the 1071.86s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0512	 = Validation score   (-root_mean_squared_erro

[备份完成] 模型已保存到: model/autogluon_model_backup
======== 第 1 段训练结束，并已备份到 Drive ========

======== 第 2/6 段训练开始 ========
使用 fit_extra 继续训练...


	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0488	 = Validation score   (-root_mean_squared_error)
	1025.03s	 = Training   runtime
	36.34s	 = Validation runtime
Fitting model: LightGBM_19_BAG_L1 ... Training model for up to 2566.7s of the 2566.7s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0456	 = Validation score   (-root_mean_squared_error)
	145.72s	 = Training   runtime
	2.47s	 = Validation runtime
Fitting model: RandomForestMSE_8_BAG_L1 ... Training model for up to 2418.94s of the 2418.94s of remaining time.
	-1.0472	 = Validation score   (-root_mean_squared_error)
	538.43s	 = Training   runtime
	32.24s	 = Validation runtime
Fitting model: CatBoost_8_BAG_L1 ... Training model for up to 1847.39s of the 1847.39s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0497	 = Validation score   (-root_mean_squared_erro

...

[备份完成] 模型已保存到: model/autogluon_model_backup
======== 第 3 段训练结束，并已备份到 Drive ========

======== 第 4/6 段训练开始 ========
使用 fit_extra 继续训练...


	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0488	 = Validation score   (-root_mean_squared_error)
	997.24s	 = Training   runtime
	36.62s	 = Validation runtime
Fitting model: LightGBM_21_BAG_L1 ... Training model for up to 2597.61s of the 2597.6s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0456	 = Validation score   (-root_mean_squared_error)
	145.86s	 = Training   runtime
	2.49s	 = Validation runtime
Fitting model: RandomForestMSE_10_BAG_L1 ... Training model for up to 2449.76s of the 2449.76s of remaining time.
	-1.0472	 = Validation score   (-root_mean_squared_error)
	540.78s	 = Training   runtime
	31.93s	 = Validation runtime
Fitting model: CatBoost_10_BAG_L1 ... Training model for up to 1876.22s of the 1876.22s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0497	 = Validation score   (-root_mean_squared_er

[备份完成] 模型已保存到: model/autogluon_model_backup
======== 第 5 段训练结束，并已备份到 Drive ========

======== 第 6/6 段训练开始 ========
使用 fit_extra 继续训练...


Excluded models: ['KNN'] (Specified by `excluded_model_types`)
Fitting 9 L1 models ...
Fitting model: LightGBMXT_24_BAG_L1 ... Training model for up to 3600.0s of the 3600.0s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0488	 = Validation score   (-root_mean_squared_error)
	1069.08s	 = Training   runtime
	36.86s	 = Validation runtime
Fitting model: LightGBM_23_BAG_L1 ... Training model for up to 2525.35s of the 2525.35s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0456	 = Validation score   (-root_mean_squared_error)
	148.43s	 = Training   runtime
	2.46s	 = Validation runtime
Fitting model: RandomForestMSE_12_BAG_L1 ... Training model for up to 2374.92s of the 2374.91s of remaining time.
	-1.0472	 = Validation score   (-root_mean_squared_error)
	541.81s	 = Training   runtime
	32.2s	 = Validation runtime
Fitting model: CatBoost_12_BAG_L1 ... Training model 

[备份完成] 模型已保存到: model/autogluon_model_backup
======== 第 6 段训练结束，并已备份到 Drive ========


# 4、检查模型

In [18]:

# 判断是否完成训练
prev_models = set(predictor.get_model_names())
predictor.fit_extra(
    hyperparameters='default',
    time_limit=60,
)
new_models = set(predictor.get_model_names())
if prev_models == new_models:
    print("✅ No new models trained — training is complete.")
else:
    print("🔁 New models added — continuing training.")

Fitting 11 L1 models ...
Fitting model: KNeighborsUnif_BAG_L1 ... Training model for up to 60.0s of the 60.0s of remaining time.
	-1.184	 = Validation score   (-root_mean_squared_error)
	5.64s	 = Training   runtime
	9.81s	 = Validation runtime
Fitting model: KNeighborsDist_BAG_L1 ... Training model for up to 44.06s of the 44.05s of remaining time.
	-1.2142	 = Validation score   (-root_mean_squared_error)
	5.54s	 = Training   runtime
	9.39s	 = Validation runtime
Fitting model: LightGBMXT_25_BAG_L1 ... Training model for up to 28.67s of the 28.67s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	-1.0644	 = Validation score   (-root_mean_squared_error)
	26.01s	 = Training   runtime
	2.56s	 = Validation runtime
Fitting model: LightGBM_24_BAG_L1 ... Training model for up to 0.02s of the 0.02s of remaining time.
	Fitting 10 child models (S1F1 - S1F10) | Fitting with ParallelLocalFoldFittingStrategy
	Time limit exceeded... Skipping Li

🔁 New models added — continuing training.
